In [9]:
from konlpy.tag import Okt
import networkx as nx
import numpy as np
from itertools import combinations
from collections import Counter

In [10]:
import pandas as pd
from tqdm import tqdm
from multiprocessing import Pool, cpu_count

In [11]:
okt = Okt()

In [12]:
def tokenize(text):
    # 형태소 분석을 통해 명사만 추출
    return [word for word in okt.nouns(text) if len(word) > 1]

In [39]:
def extract_keywords(text, num_keywords=20):
    okt = Okt()
    
    # 형태소 분석 및 명사 추출
    words = tokenize(text)
    
    # 단어들의 출현 빈도수 계산
    word_counts = Counter(words)
    
    # 그래프 생성
    graph = nx.Graph()
    
    # 단어 노드 추가
    for word, count in word_counts.items():
        if count > 1:  # 최소 출현 빈도 조건
            graph.add_node(word, count=count)
    
    # 단어의 연결을 위한 윈도우 크기 설정
    window_size = 4
    
    # 윈도우 안에서 단어 간의 연결 설정
    for i in range(len(words) - window_size + 1):
        window_words = words[i:i + window_size]
        for w1, w2 in combinations(window_words, 2):
            if graph.has_node(w1) and graph.has_node(w2):
                if graph.has_edge(w1, w2):
                    graph[w1][w2]['weight'] += 1
                else:
                    graph.add_edge(w1, w2, weight=1)
    
    # PageRank 계산
    rank = nx.pagerank(graph, weight='weight')
    
    # 상위 num_keywords개의 키워드 추출
    top_keywords = sorted(rank.items(), key=lambda x: x[1], reverse=True)[:num_keywords]
    
    return top_keywords

    #return [keyword for keyword, _ in top_keywords]

In [ ]:
directory_path = '/home/osung/data/korean/modu/json'
df = pd.read_csv(directory_path+'/combined_news.tsv', sep='\t')

In [14]:
len(df)

3611424

In [15]:
df.columns

Index(['id', 'topic', 'text'], dtype='object')

In [18]:
df.iloc[-1]

id                                     NZRW2200000001.2270
topic                                                   정치
text     TBS예산 격론 끝에 ‘55억원 삭감’ 의결 서울시의회가 44조 원 규모의 2022...
Name: 3611423, dtype: object

In [19]:
doc = df.iloc[0].text

In [20]:
doc

'"대통령, 시장 방문만 하지 말고 실천해달라" 2008년의 마지막 새벽, 언론의 카메라는 서울 여의도를 향했다. 방송법 등 주요쟁점 법안이 상정될 국회 본회의장을 두고 여야 의원들의 전쟁을 기다리고 있었던 것. 같은 시각, 국회 밖 세상에서 서민들은 경제 위기와 강추위 속에서 삶의 고단함과 정치에 대한 절망감에 맞선 채 팍팍한 삶을 이어가고 있었다. 이들의 목소리를 듣기 위해 이날 새벽 3시 대한민국의 아침을 여는 서울 가락동 농수산물종합도매시장으로 향했다. 가락시장으로 가는 택시 안에서 2008년의 마지막 새벽을 맞는 서민들의 얘기를 엿들을 수 있었다. 택시기사 서인철(가명·63)씨는 말한다. "LPG값이 휘발유보다 비싸졌고, 9만9000원 하는 사납금 내기도 벅차다. (한 달에) 100만원도 벌기 힘들다. 40년 동안 택시기사를 했는데, 요즘이 제일 힘들다. 택시 손님 열에 아홉은 힘들다고 한다. 더 암울한 건 내년엔 더 어렵다는 거다. 정치인들? 멱살 잡고 싸우는 거 말고 뭘 하나? 이젠 짜증내기도 귀찮다." 치열한 삶터, 가락시장에 드리워진 불황의 그림자 새벽 3시 30분, 가락시장의 첫인상은 그야말로 치열한 삶터였다. 가락시장 인근 도로엔 농수산물을 싣고 전국에서 올라온 트럭들이 \'차산차해\'를 이뤘다. 가락시장 안에는 트럭과 함께 짐을 옮기는 지게차·오토바이·손수레로 가득 차 더욱 복잡했다. 가락시장 수산시장에서는 방어 경매가 한창이었다. 중개인이 일반인들이 알아듣기 힘든 말로 몇 차례 중얼거리자 경매는 이내 끝났다. 방어는 어찌나 힘이 좋은지 한 번 몸을 뒤집자 바구니 밖 3~4m까지 물이 튈 정도였다. 상인들도 방어만큼이나 바삐 움직였다. 사람들은 분주히 수산물을 날랐다. 경매로 수산물을 산 중간도매상들은 좌판을 펼쳐놓고 손님들을 기다렸다. 그들은 머리엔 모자·스카프 등을 두르고, 신발은 털장화를 싣고는 난로 옆에 바싹 붙어 생선 손질을 하고 있었다. 쪼그리고 앉아 고등어 배를 가르고 있던 김진숙(가명·51)씨의 표정엔 노곤함이 묻어났다. 전

In [28]:
keywords = extract_keywords(doc)

In [29]:
keywords

[('가락시장', 0.03646997254429091),
 ('사람', 0.03454764351556988),
 ('수산시장', 0.03405599497941906),
 ('새벽', 0.03111153536702474),
 ('시장', 0.029838413039404724),
 ('배추', 0.028920909699708394),
 ('서민', 0.028031749849944568),
 ('경제', 0.02661029094416228),
 ('상인', 0.025143270050378596),
 ('가명', 0.024605998059264184)]

In [32]:
keywords[0][1]

0.03646997254429091

In [34]:
df1 = df[:100]

In [35]:
df1

,id,topic,text
0,NIRW1900000001.1,사회,"""대통령, 시장 방문만 하지 말고 실천해달라"" 2008년의 마지막 새벽, 언론의 카..."
1,NIRW1900000001.4,사회,"진성호 의원, 겨우 이 정도였나 오늘(31일) 한나라당 의원 총회에서 조선일보 출신..."
2,NIRW1900000001.5,사회,"""소의 해, 정치인들 싸움 좀 그만하시죠"" 지난해 12월 31일 대구국채보상기념운동..."
3,NIRW1900000001.6,사회,'MB악법 저지' 촛불 들고 새해 맞은 대전시민들 촛불의 물결이 전국을 뒤 덮었던 ...
4,NIRW1900000001.7,사회,산(酸) 몰아내니 누이 좋고 매부 좋고 겨울철 우리의 입맛을 돋우는 먹을거리 가운데...
...,...,...,...
95,NIRW1900000001.198,정치,"민주당, 12일만에 본회의장 농성 해제 민주당 12일만에 본회의장 농성 해제 민주당..."
96,NIRW1900000001.201,사회,"천안시 부당 행정 무더기 적발 계약 체결 부적정, 공사 설계 부적정, 보조금 정산검..."
97,NIRW1900000001.203,사회,일부 반대 속 안양 만안뉴타운 지원센터 열어 안양시와 경기도시공사는 만안 뉴타운사업...
98,NIRW1900000001.204,사회,2009년의 길목에서 연말을 보내고 새해를 맞이하면서도 대다수 사람들의 마음은 그리...


In [36]:
documents = df1.text

In [40]:
doc_keywords = []

for doc in tqdm(documents) :
    keywords = extract_keywords(doc)
    doc_keywords.append(keywords)

100%|█████████████████████████████████████████| 100/100 [00:43<00:00,  2.28it/s]


In [41]:
doc_keywords

[[('가락시장', 0.03646997254429091),
  ('사람', 0.03454764351556988),
  ('수산시장', 0.03405599497941906),
  ('새벽', 0.03111153536702474),
  ('시장', 0.029838413039404724),
  ('배추', 0.028920909699708394),
  ('서민', 0.028031749849944568),
  ('경제', 0.02661029094416228),
  ('상인', 0.025143270050378596),
  ('가명', 0.024605998059264184),
  ('채소', 0.023819859082242937),
  ('도매', 0.02289853855588062),
  ('수산물', 0.022429575180303484),
  ('택시', 0.019913574914715182),
  ('중간', 0.019386854184506443),
  ('달라', 0.017582973506089887),
  ('작년', 0.017411372076471376),
  ('가격', 0.01630961155403351),
  ('하루', 0.01620970435601311),
  ('생선', 0.01618935790918644)],
 [('의원', 0.13532769889385685),
  ('최소한', 0.06481192311558978),
  ('망언', 0.055133128833688726),
  ('국민', 0.052760215027543154),
  ('인간', 0.04816740467927247),
  ('우리', 0.04789356008898691),
  ('품위', 0.045652982842999464),
  ('대한', 0.04293368732321917),
  ('국회의원', 0.04252054351407282),
  ('라면', 0.042297717215555634),
  ('문제', 0.036837770574140304),
  ('출신', 0.036

In [42]:
pip install pandas nltk

/bin/bash: /home/osung/anaconda3/envs/diffusion/lib/libtinfo.so.6: no version information available (required by /bin/bash)
Note: you may need to restart the kernel to use updated packages.


In [59]:
cpu_count() / 2

48.0

In [60]:
len(doc_keywords)

1190

In [69]:
keyword_lists = parallel_keyword_extraction(documents[:10], num_keywords=10)

  0%|                                                    | 0/10 [00:00<?, ?it/s]


AssertionError: phrase input should be string, not <class 'tuple'>

In [44]:
import multiprocessing as mp

In [45]:
# 텍스트 데이터 나누기
def chunk_dataframe(df, chunk_size):
    return [df[i:i + chunk_size] for i in range(0, df.shape[0], chunk_size)]

In [46]:
# 병렬 처리 함수
def parallelize_dataframe(df, func, num_cores=4):
    df_split = chunk_dataframe(df, chunk_size=int(df.shape[0] / num_cores))
    pool = mp.Pool(num_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [47]:
def extract_keywords_chunk(chunk, num_keywords=20):
    documents = chunk['text']
    doc_keywords = []

    for doc in tqdm(documents) :
        keywords = extract_keywords(doc)
        doc_keywords.append(keywords)
        
    chunk['keywords'] = doc_keywords
    
    return chunk

In [50]:
num_cores = 4 #mp.cpu_count()

In [51]:
num_cores

4

In [53]:
df_with_keywords = parallelize_dataframe(df[:100], extract_keywords_chunk, num_cores=num_cores)

  4%|█▊                                          | 1/25 [00:00<00:08,  2.72it/s]

KeyboardInterrupt: 